In [ ]:
import numpy as np
from scipy.optimize import fsolve
from scipy.integrate import quad

#Input Values

Now, we need to give our input conditions and the constants that we are going to use.

Paper conditions are:

| Description| Unit| High Pressure (Point 11-12')| Intermediate Pressure (Point 13-18) |Low Pressure (Point 15-16) |
|----------|----------|----------|----------|----------|
|   Steam Inlet Pressure | bar    | 300     | 83.07        | 11.99        |
|   Steam Inlet Temperature | °C  | 600     | 600          |  600        |
|   Steam Outlet Pressure | bar   | 83.07   | 11.99         | 0.10        |
|   Steam Outlet Temperature | °C | 395.5   | 320.1        | 73.35        |

\

| Description| Unit| High Pressure (Point 11-12')| Intermediate Pressure (Point 13-18) |Low Pressure (Point 15-16) |
|----------|----------|----------|----------|----------|
|   Steam Inlet Enthalpy |  kJ/kg   | 3443   | 3639 | 3697  |
|   Steam Outlet Enthalpy |  kJ/kg  | 3119   | 3089 | 2636  |


## Report Input Values

In [ ]:
### Input conditions from Report:

# # High pressure (11 -> 12)
# P1 = 10 # in bar
# T1 = 1200 # in Celsius
# P2 = 4 # in bar
# T2 = 1100 # in Celsius

# ## Intermediate pressure (13 -> 18)
# P1 = 83.07 # in bar
# T1 = 600 # in Celsius
# P2 = 11.99 # in bar
# T2 = 320.1 # in Celsius

# # Low pressure (15 -> 16)
# P1 = 11.99 # in bar
# T1 = 600 # in Celsius
# P2 = 0.10 # in bar
# T2 = 73.35 # in Celsius


## Steam Tables Input Values

In [ ]:
### Input conditions from Steam Tables:

# # High pressure ST
# P1 = 200 # in bar
# T1 = 400 # in Celsius
# P2 = 50 # in bar
# T2 = 300 # in Celsius

# ## Intermediate pressure ST
# P1 = 10 # in bar
# T1 = 400 # in Celsius
# P2 = 4 # in bar
# T2 = 300 # in Celsius

# Low pressure ST
P1 = 10 # in bar
T1 = 1200 # in Celsius
P2 = 4 # in bar
T2 = 1100 # in Celsius

## User Input Code

In [ ]:
species = 'H2O' # enter the name of the species here

# #User Input (Commenting it out for now to test and run code more effeciently)
# P1 = input("Enter the value for inital pressure in bar: ") # in bar
# P1=int(P1)
# T1 = input("Enter the value for inital temperature in Celsius: ") # in Celsius
# T1=int(T1)

# ## final pressure and volume
# P2 = input("Enter the value for final pressure in bar: ") # in bar
# P2=int(P2)
# T2 = input("Enter the value for final temperature in Celsius: ") # in bar # in Celsius
# T2=int(T2)

# species = input("Enter name of species: H2O, N2, O2, CO2, C2H6") # enter the name of the species here

#Critical Values

Since we would be looking at different species besides water, we elected to include a prototype that will have the critical points and important properties for water. Use critical constants values from Appendix A of Koretsky.

In [ ]:
def properties(species):
    """Data for the critical temperature and pressure of a species"""
    # YOUR CODE STARTS HERE
    if species in np.array(['H2O','water']):
      T_c = 647.3 # critical temperature of Water (in Kelvin)
      P_c = 220.48e5 # critical pressure of Water (in Pascals)
      MW = 18.015e-3 #Molecular weight of Water kg/gmol
      w = 0.344 #Pitzer acentric factor of Water (important for Peng-Robinson EOS)
    elif species in np.array(['N2']):
      T_c = 126.2 #K
      P_c = 33.84e5 #Pa
      MW = 28.0134e-3 #molecular weight of N2 kg/mol
      w = 0.039 #Pitzer acentric factor
    elif species in np.array(['O2']):
      T_c = 154.6 #K
      P_c = 50.46e5 #Pa
      MW = 31.999e-3 #molecular weight of O2 kg/mol
      w = 0.021 #Pitzer acentric factor
    elif species in np.array(['CO2']):
      T_c = 304.2 #K
      P_c = 73.76e5 #Pa
      MW = 44.010e-3 #molecular weight of CO2 kg/mol
      w = 0.225 #Pitzer acentric factor
    elif species in np.array(['H2']):
      T_c = 33.2 #K
      P_c = 12.97e5 #Pa
      MW = 2.016e-3 #MW kg/mol
      w = -0.22 #Pitzer acentric factor
    elif species in np.array(['C2H6']):
      T_c = 305.4 #K
      P_c = 48.74e5 #Pa
      MW = 30.069e-3 #MW kg/mol
      w = 0.099 #Pitzer acentric factor
    elif species in np.array(['NH3']):
      T_c = 405.6 #K
      P_c = 112.77e5 #Pa
      MW = 17.031e-3 #MW kg/mol
      w = 0.250 #Pitzer acentric factor
    return (T_c, P_c, w, MW)


# Peng-Robinson Equation of S:
$$P = \frac{RT}{v-b} -\frac{aα}{v^2+2bv-b^2}$$
where
$$ a=0.45724\frac{(RTc)^2}{Pc},$$
$$ b=0.07780\frac{RTc}{Pc},$$
$$ α=[1+\kappa(1-\sqrt{\frac{T}{Tc}}]^2$$
$$ K = 0.37464 +1.54226ω - 0.26992ω^2$$

In [ ]:
print(properties(species))
def P_PR(T, v, species):
    """Compute the pressure from PR EOS"""
    # YOUR CODE STARTS HERE
    T_c, P_c, w, MW  = properties(species)
    T_K = T + 273.15 # temperature converted from Celcius to Kelvin
    R_gas = 8.314 # gas constant in J K^-1 mol^-1
    a = 0.45724 * (R_gas * T_c)**2 /P_c # units = m^6*Pa/mol^2 (Note that to get an exponent, use '**' notation )
    b = 0.07780 * R_gas * T_c / P_c # units = m^3/mol
    k = 0.37464 + 1.54226*w - 0.26992*w**2
    alpha = (1 + k * (1 - np.sqrt(T_K/T_c)) )**2

    P = ((R_gas * T_K)/(v - b)) - ((a * alpha)/(v**2 + 2 * b * v - b**2)) # in Pa
    return P

(647.3, 22048000.0, 0.344, 0.018015)


# Ideal Gas Law

$$ v = \frac{R T}{P} $$

We will need this later to determine a good guess for $v$ using the given conditions.

In [ ]:
def v_ideal(P,T):
    """Molar volume from the ideal gas law"""
    R_gas = 8.314 # gas constant in J K^-1 mol^-1
    P_Pa = P * 1e5 # convert bar to Pa
    T_K = 273.15 + T
    v = R_gas * T_K / P_Pa # in m^3/mol
    return v

# Solving for the Molar Volume


In [ ]:
def PR_v_func(v, P, T, species = species): # While writing equation of state functions, follow the sequenece: v,P,T
    # YOUR CODE STARTS HERE
    RHS = P_PR(T, v, species) # pressure from PR EOS, given the T and v
    LHS = P*1e5 # given pressure in Pa
    return LHS-RHS

Now we will solve for $v$ using PR
 EOS. We will use the $v$ from the ideal gas law as a guess

In [ ]:
args1 = (P1,T1) # additional arguments for the function
                # (NOTE: The sequence of writing the additional arguments
                # should be same as you have written in function defination)

v01 = v_ideal(*args1) # initial guess for volume
                      # (NOTE: It is important to use correct initial guess
                      # to get the volume. For starters, use Ideal gas volume
                      # as an initial guess)

# volume for state 1
v1= fsolve(PR_v_func,v01,args1)

print(v1)

# volume for state 2
args2 = (P2,T2)
v02 = v_ideal(*args2)
v2 = fsolve(PR_v_func,v02,args2)

[0.01225168]


# Print Results

We will need to print $v$. Here is a code that will do it (no need to change it):

In [ ]:
def print_specific_volume(v, T, p, species):
    """Print the specific volume, temperature, and pressure of the species"""
    T_c, P_c, w, MW  = properties(species)
    print(f'Results for {species}:')
    num_lin = 74 * '-'
    print(num_lin)
    print('| State | Temperature (C) | Pressure (bar) | Specific Volume (m^3 kg^-1) |')
    print(num_lin)
    for l in range(T.shape[0]):
        print(f'|{l + 1:^7.0f}|{T[l]:^17.2f}|{p[l]:^16.2f}|{v[l]/MW:^29.6f}|')
    print(num_lin)
    return None

v_array = np.array([v1[0], v2[0]])
T_array = np.array([T1, T2])
p_array = np.array([P1, P2])
print_specific_volume(v_array, T_array, p_array, species)

Results for H2O:
--------------------------------------------------------------------------
| State | Temperature (C) | Pressure (bar) | Specific Volume (m^3 kg^-1) |
--------------------------------------------------------------------------
|   1   |     1200.00     |     10.00      |          0.680082           |
|   2   |     1100.00     |      4.00      |          1.584287           |
--------------------------------------------------------------------------


Check your results with the values in Steam tables. They are matching, aren't they?

# Integrating Heat Capacities

In [ ]:
import numpy as np
from scipy.optimize import fsolve  # Solves for one unknown if one equation is given
from scipy.integrate import quad  # Solver that is used for integration

# Creating parameters for calculating heat capacities
def CpParams(species):
    if species in np.array(['H2O', 'water']):
        A = 3.470
        B = 0.00145
        C = 0.0
        D = 12100
        E = 0.0
    elif species in np.array(['N2']):
        A = 3.280
        B = 0.000593
        C = 0.0
        D = 4000
        E = 0.0
    elif species in np.array(['O2']):
        A = 3.639
        B = 0.000506
        C = 0.0
        D = -22700
        E = 0.0
    elif species in np.array(['CO2']):
        A = 5.457
        B = 0.001045
        C = 0.0
        D = -115700
        E = 0.0
    elif species in np.array(['C2H6']):
        A = 1.131
        B = 0.019225
        C = -0.000005561
        D = 0.0
        E = 0.0
    elif species in np.array(['H2']):
        A = 3.639
        B = 0.000422
        C = 0.0
        D = -8300
        E = 0.0
    elif species in np.array(['NH3']):
        A = 3.5778
        B = 0.00302
        C = 0.0
        D = -18600
        E = 0.0
    return (A, B, C, D, E)

# Defining a function to calculate the heat capacities
A, B, C, D, E = CpParams(species)
def CpCalc(T, A, B, C, D, E):
    T = T + 273.15 # need to convert celcius to kelvin
    R = 0.008314  # the gas constant in units of Kj mol K^-1
    Cp = R * (A + (B * T) + (C * T**2) + (D * (T**(-2))) + (E * (T**3)))
    return Cp

h_ig, _ = quad(CpCalc,T1,T2, args = (A, B, C, D, E))
h_ig = h_ig *1000
print('The integrated Cp is', h_ig, 'J mol^-1 for',species)


The integrated Cp is -4605.581156681005 J mol^-1 for H2O


# Departure functions for the PR EOS
Departure functions:

$$\displaystyle \Delta h' (T,P) = - \int_\infty ^v \left[ T \left(\frac{\partial P}{\partial T}\right)_v +v \left(\frac{\partial P}{\partial v}\right)_T \right] dv$$

$$\displaystyle \Delta s' (T,P) = - \int_\infty^v \left[
 \left( \frac{\partial P}{\partial T} \right)_v - \left(\frac{R}{v}\right) \right] dv - R \ln \left( \frac{Pv}{RT}\right)$$

Using the PR EOS, we can derive $\left( \frac{\partial P}{\partial T} \right)_v$ and $\left(\frac{\partial P}{\partial v}\right)_T$.

They are:
$$ \left( \frac{\partial P}{\partial T} \right)_v  = \frac{R}{v - b}$$
$$ \left(\frac{\partial P}{\partial v}\right)_T = -\left( \frac{2aα(b+v)}{(v^2+2bv-b^2)^2} - \frac{RT}{(b-v)^2} \right)$$

We will solve for $\Delta h' (T,P)$ and $\Delta s' (T,P)$:

In [ ]:
def dhprim_integrand(v, T, species):
    """Integrand for the Departure Function for enthalpy"""
    T_c, P_c, w, MW = properties(species)
    R_gas = 8.314
    a = 0.45724 * (R_gas * T_c)**2 / P_c
    b = 0.07780 * R_gas * T_c / P_c
    T_r = T / T_c #reduced temperature
    k = 0.37464 + 1.54226 * w - 0.26992 * w**2
    alpha = (1 + k * (1 - np.sqrt(T_r)))**2

    #derivatives
    dalpha_dT = -k * np.sqrt(alpha / (T_r)) / T_c
    dP_dT = R_gas / (v - b) - dalpha_dT * (a / (v**2 + 2 * v * b - b**2))
    dP_dv = - (R_gas * T / (v - b)**2 - (2 * a * alpha * (v + b)) / (v**2 + 2 * v * b - b**2)**2)

    #integrand
    integrand = dP_dT * T + dP_dv * v  # in J/mol
    return integrand

def dsprim_integrand(v, T, species):
    """Integrand for the Departure Function for entropy"""
    T_c, P_c, w, MW = properties(species)
    R_gas = 8.314
    a = 0.45724 * (R_gas * T_c)**2 / P_c
    b = 0.07780 * R_gas * T_c / P_c
    T_r = T / T_c #reduced temperature
    k = 0.37464 + 1.54226 * w - 0.26992 * w**2
    alpha = (1 + k * (1 - np.sqrt(T_r)))**2

    #derivatives
    dalpha_dT = -k * np.sqrt(alpha / (T_r)) / T_c
    dP_dT = R_gas / (v - b) - dalpha_dT * (a / (v**2 + 2 * v * b - b**2))

    #integrand
    integrand = dP_dT - R_gas / v  # in J/mol
    return integrand

def dh_ds_prim(v, T, P, species):
    """Departure function values for dh' and ds'"""
    T_K = T + 273.15
    P_Pa = P * 1e5  # in Pa
    arguments = (T_K, species)
    R_gas = 8.314

    #Dh departure function
    dh_prime = -quad(dhprim_integrand, v, np.inf, args=arguments)[0]

    #Ds departure function
    ds_integral = quad(dsprim_integrand, v, np.inf, args=arguments)[0]
    ds_prime = -ds_integral + R_gas * np.log(P_Pa * v / (R_gas * T_K))
    return (dh_prime, ds_prime)


# Print the results

In [ ]:
def partion_func_results(v, T, p, species):
    """Print results for the entropy and enthalpy partion functions"""
    h_array=[]
    s_array=[]
    print(f'Results for {species}:')
    num_lin = 48 * '-'
    print(num_lin)
    print("| State | Dh' (J mol-1) | Ds' (J  mol^-1 K^-1) |")
    print(num_lin)
    for l in range(T.shape[0]):
        h, s = dh_ds_prim(v[l], T[l], p[l], species)
        h_array += h,
        s_array += s,
        print(f'|{l + 1:^7.0f}|{h:^15.3f}|{s:^21.3f}|')
    print(num_lin)
    return (np.array(h_array), np.array(s_array))
h_array,s_array = partion_func_results(v_array, T_array, p_array, species)


Results for H2O:
------------------------------------------------
| State | Dh' (J mol-1) | Ds' (J  mol^-1 K^-1) |
------------------------------------------------
|   1   |    -47.058    |       -0.035        |
|   2   |    -23.691    |       -0.017        |
------------------------------------------------


# Finding the entropy change from state 1 to state 2:

$$\displaystyle  s_2 - s_1 = \int_{T_1}^{T_2}
 \left( \frac{Cp}{T} \right) dT - R \ln \left( \frac{P_2}{P_1}\right)$$

# the integral will be labeled "integral" in the code, while the -Rln will be labeled "constant"

In [ ]:
def Cp_TdT(T1, T2, species):
    """Calculating integrated Cp divided by T"""
    R_gas = 8.314
    if species in np.array(['N2']):
      coeff = np.array([3.280, 0.593e-3,0.04e5])
    elif species in np.array(['H2O', 'water']):
      coeff = np.array([3.470, 1.45e-3, 0.121e5])
    integration = R_gas * (coeff[0] * np.log(T2/T1) + coeff[1] * (T2 - T1) - (coeff[2] / 2) * (1/ T2**2 - 1/T1**2))
    return integration

T1K = T1 + 273.15
T2K = T2 + 273.15
integration = Cp_TdT(T1K, T2K, species)
entropychange = -s_array[0] + integration + s_array[1]

print('The entropy change from state 1 to state 2 is ', entropychange, 'J K^-1 mol^-1')

The entropy change from state 1 to state 2 is  -3.219701128718816 J K^-1 mol^-1


# Isentropic Process

## Finding the exit temperature if the process were isentropic:

 $$\displaystyle  \int_{T_1}^{T_2 (isentropic)}
 \left( \frac{Cp}{T} \right) dT = R \ln \left( \frac{P_2}{P_1}\right)$$

 # We will be solving for the upper bound variable:
 $$\displaystyle T_2 (isentropic) = $$

In [ ]:
def s_p(P1, P2):
    R_gas = 8.314
    s = R_gas * np.log(P2/P1)
    return s

def solve_isentropic(T2, T1, P1, P2, species):
    T1K = T1 + 273.15
    T2K = T2 + 273.15
    lhs = Cp_TdT(T1K, T2K, species)
    rhs = s_p(P1, P2)
    return lhs - rhs

T_trial = T_array[1]
#print(T_trial)
arguments = (T_array[0], *p_array, species)
P1 = arguments[1]
P2 = arguments[2]

#print('T array values:', T_array[0], T_array[1])
#print(arguments[0], arguments[1], arguments[2], arguments[3])
#print(s_p(P1, P2))

T_isentropic = fsolve(solve_isentropic, T_trial, arguments)[0]

print(f'Isentropic exit temperature for {species}: {T_isentropic:0.2f} C')

Isentropic exit temperature for H2O: 971.74 C


##Isentropic Entropy and Enthalpy

In [ ]:
T2 = T_isentropic
args2 = (P2,T2)
v02 = v_ideal(*args2)
v2 = fsolve(PR_v_func,v02,args2)

v_array = np.array([v1[0], v2[0]])
T_iso_array = np.array([T1, T2])
p_array = np.array([P1, P2])

h_iso_ig, _= quad(CpCalc,T_iso_array[0],T_iso_array[1], args = (A, B, C, D, E))
h_iso_ig = h_iso_ig * 1000
print('The new integrated Cp from T1 to the isentropic exit temperature is',h_iso_ig, 'J mol^-1')

print_specific_volume(v_array, T_iso_array, p_array, species)

h_iso_array, s_iso_array = partion_func_results(v_array, T_iso_array, p_array, species)


The new integrated Cp from T1 to the isentropic exit temperature is -10337.431689939742 J mol^-1
Results for H2O:
--------------------------------------------------------------------------
| State | Temperature (C) | Pressure (bar) | Specific Volume (m^3 kg^-1) |
--------------------------------------------------------------------------
|   1   |     1200.00     |     10.00      |          0.680082           |
|   2   |     971.74      |      4.00      |          1.435949           |
--------------------------------------------------------------------------
Results for H2O:
------------------------------------------------
| State | Dh' (J mol-1) | Ds' (J  mol^-1 K^-1) |
------------------------------------------------
|   1   |    -47.058    |       -0.035        |
|   2   |    -31.346    |       -0.023        |
------------------------------------------------


# Creating a code to solve for the work of the turbine:

$$ \displaystyle  w_{turbine} = \Delta h_{rg} = - \Delta h_2 + \Delta h_1 + \Delta h_{ig} $$

In [ ]:
def w_turbine(diff_h1_prim, diff_h2_prim, diff_ig):
    """Calculate the work of a turbine"""
    w_t = (- diff_h1_prim + diff_ig + diff_h2_prim)
    return -w_t

def print_w_turbine(T, h1_prim, h2_prim, diff_ig, species):
    """Print the integrated ideal heat capacity"""
    print(f'Results for {species}:')
    num_lin = 47 * '-'
    print(num_lin)
    print("| T_1 (C) | T_2 (C) |  Turbine Work (KJ/mol)  |")
    print(num_lin)
    T_1 = T[0] + 273.15
    T_2 = T[1] + 273.15
    w_t = w_turbine(h1_prim, h2_prim, diff_ig)
    print(f'|{T[0]:^9.2f}|{T[1]:^9.2f}|{w_t/1000:^25.3f}|')
    print(num_lin)
    return w_t

w_t = print_w_turbine(T_array, *h_array, h_ig, species)

Results for H2O:
-----------------------------------------------
| T_1 (C) | T_2 (C) |  Turbine Work (KJ/mol)  |
-----------------------------------------------
| 1200.00 | 1100.00 |          4.582          |
-----------------------------------------------


# Calculate isentropic work:

In [ ]:
def w_turbine(diff_h1_prim, diff_h2_prim, diff_ig):
    """Calculate the work of a turbine"""
    w_t = (- diff_h1_prim + diff_ig + diff_h2_prim)
    return -w_t

def print_w_turbine(T, h1_prim, h2_prim, diff_ig, species):
    """Print the integrated ideal heat capacity"""
    print(f'Results for {species}:')
    num_lin = 47 * '-'
    print(num_lin)
    print("| T_1 (C) | T_2 (C) |  Turbine Work (KJ/mol)  |")
    print(num_lin)
    T_1 = T[0] + 273.15
    T_2 = T[1] + 273.15
    w_t = w_turbine(h1_prim, h2_prim, diff_ig)
    print(f'|{T[0]:^9.2f}|{T[1]:^9.2f}|{w_t/1000:^25.3f}|')
    print(num_lin)
    return w_t

w_t_2 = print_w_turbine(T_iso_array, *h_iso_array, h_iso_ig, species)

Results for H2O:
-----------------------------------------------
| T_1 (C) | T_2 (C) |  Turbine Work (KJ/mol)  |
-----------------------------------------------
| 1200.00 | 971.74  |         10.322          |
-----------------------------------------------


#Efficiency

In [ ]:
#Efficiency
efficiency = w_t/w_t_2
print(f'The efficiency is {efficiency*100:0.2f}%.')

The efficiency is 44.39%.
